## Notes 
- For the moment the model is only tokenized with the `text_body` and not with the `text_title`
- The evalutation training metric is the `accuracy`. Is it the best metric for an ordered variable ? (classify 4 instead of 5 is less important than classify 1 instead of 5)
- for the moment we do not monitor the evaluation metric during fine-tuning. What does it mean ? Is it useful ? 


In [1]:
import torch
print(torch.cuda.is_available())

print(torch.cuda.device_count())
print(torch.cuda.current_device())
torch.cuda.device(0)
torch.cuda.get_device_name(0)

True
2
0


'NVIDIA TITAN Xp'

In [2]:
model_fr_name = 'camembert-base'
model_en_name = 'roberta-base'
model_translation = {}
model_translation['fr_en'] = 'Helsinki-NLP/opus-mt-fr-en'
model_translation['en_fr'] = 'Helsinki-NLP/opus-mt-en-fr'
dataset_name = 'amazon_reviews_multi'

In [3]:
from datasets import load_dataset

dataset = load_dataset(dataset_name)

No config specified, defaulting to: amazon_reviews_multi/all_languages
Found cached dataset amazon_reviews_multi (/home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from datasets.dataset_dict import DatasetDict

def stars_into_labels(example):
    example['stars'] = example['stars']-1
    return example

def get_dataset_language(full_dataset, language) :
    dataset_out = {}
    for split in full_dataset : 
        dataset_out[split] = dataset[split].filter(lambda exemple : exemple['language'] == language)
        dataset_out[split] = dataset_out[split].remove_columns(["review_id","product_id", "reviewer_id",
        "review_title","language","product_category"])
        dataset_out[split] = dataset_out[split].rename_column("review_body", "text")
        dataset_out[split] = dataset_out[split].map(stars_into_labels)
        dataset_out[split] = dataset_out[split].rename_column("stars", "label")
    return DatasetDict(dataset_out)
dataset_fr = get_dataset_language(dataset, 'fr')
dataset_en = get_dataset_language(dataset, 'en')

Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-abef70f0f0f656cb.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-6d9e53f0e04bc9b0.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-faeec778108fe3de.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-2fd593a97b59a6c3.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a

## 2.1 Fine-tune French Model

In [5]:
# Get the text tokenized

from transformers import AutoTokenizer

tokenizer = {}
tokenizer['fr'] = AutoTokenizer.from_pretrained(model_fr_name)
tokenizer['en'] = AutoTokenizer.from_pretrained(model_en_name)

def tokenize_function(examples, language):
    return tokenizer[language](examples["text"], 
                     padding="max_length", truncation=True)

for split in dataset_fr :
    dataset_fr[split] = dataset_fr[split].map(lambda examples : tokenize_function(examples,'fr'), batched=True)
    dataset_en[split] = dataset_en[split].map(lambda examples : tokenize_function(examples,'en'), batched=True)
# tokenized_en_train = dataset_en_train.map(tokenize_function, batched=True)
# tokenized_en_test = dataset_en_test.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-57dd87682cc54bc5.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-f15d5c0fcd89bef7.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-7fa0d576a9956dcd.arrow


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-9a54b48d0a4f3f20.arrow


In [6]:
#Take only a small part of the dataset

small_fr_dataset = {}
for split in dataset_fr :
    small_fr_dataset[split] =  dataset_fr[split].shuffle(seed=42).select(range(200))
small_fr_dataset = DatasetDict(small_fr_dataset)

Loading cached shuffled indices for dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-fdcfd857639c613e.arrow
Loading cached shuffled indices for dataset at /home/desponds/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609/cache-0aa473b6d1a47dee.arrow


In [7]:
from transformers import AutoModelForSequenceClassification
model = {}
model['fr'] = AutoModelForSequenceClassification.from_pretrained(model_fr_name, num_labels=5)
model['en'] = AutoModelForSequenceClassification.from_pretrained(model_en_name, num_labels=5)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 

In [8]:
from transformers import TrainingArguments, Trainer
training_args = {}
training_args['fr'] = TrainingArguments(output_dir="test_trainer_fr", evaluation_strategy="epoch")
training_args['en'] = TrainingArguments(output_dir="test_trainer_en", evaluation_strategy="epoch")
# If we want to monitor the evaluation metric during fine-tuning
# training_args = TrainingArguments(output_dir="test_trainer_fr", evaluation_strategy="epoch")

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = {}
trainer['fr'] = Trainer(
    model=model['fr'],
    args=training_args['fr'],
    train_dataset= dataset_fr['train'],
    eval_dataset= dataset_fr['validation'],
    compute_metrics=compute_metrics,
)
trainer['en'] = Trainer(
    model=model['en'],
    args=training_args['en'],
    train_dataset= dataset_en['train'],
    eval_dataset= dataset_en['validation'],
    compute_metrics=compute_metrics,
)

In [10]:
trainer['fr'].train('test_trainer_fr/checkpoint-37500')

Loading model from test_trainer_fr/checkpoint-37500.
The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 37500
  Number of trainable parameters = 110625797
  Continuing training from checkpoint, will skip to saved glo

0it [00:00, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




Epoch,Training Loss,Validation Loss


TrainOutput(global_step=37500, training_loss=0.0, metrics={'train_runtime': 0.2395, 'train_samples_per_second': 2505732.578, 'train_steps_per_second': 156608.286, 'total_flos': 1.578708854784e+17, 'train_loss': 0.0, 'epoch': 3.0})

In [12]:
trainer['en'].train('test_trainer_en/checkpoint-24500')

Loading model from test_trainer_en/checkpoint-24500.
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 37500
  Number of trainable parameters = 124649477
  Continuing training from checkpoint, will skip to saved global_

  0%|          | 0/12000 [00:00<?, ?it/s]

/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
2,0.889900,0.952181,0.599800
3,0.762600,0.953503,0.600800


Saving model checkpoint to test_trainer_en/checkpoint-25000
Configuration saved in test_trainer_en/checkpoint-25000/config.json
Model weights saved in test_trainer_en/checkpoint-25000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars;

TrainOutput(global_step=37500, training_loss=0.2767451969401042, metrics={'train_runtime': 8012.444, 'train_samples_per_second': 74.884, 'train_steps_per_second': 4.68, 'total_flos': 1.578708854784e+17, 'train_loss': 0.2767451969401042, 'epoch': 3.0})

## Translation

SOhelp : https://stackoverflow.com/questions/70043467/how-to-run-huggingface-helsinki-nlp-models

In [18]:
# Try 1 of traduction
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_trad = AutoTokenizer.from_pretrained(model_translation['fr_en'] )
model_trad = AutoModelForSeq2SeqLM.from_pretrained(model_translation['fr_en'] )

inp = "Je m'appelle Mathieu et je vis à Lausanne"
input_ids = tokenizer_trad(inp, return_tensors="pt").input_ids
outputs = model_trad.generate(input_ids=input_ids, num_return_sequences=3)
print("Generated:", tokenizer_trad.batch_decode(outputs, skip_special_tokens=True))

loading configuration file config.json from cache at /home/desponds/.cache/huggingface/hub/models--Helsinki-NLP--opus-mt-fr-en/snapshots/49463f1706007cb314a942296b77a6483e6f6953/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-fr-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "decoder_vocab_size": 59514,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkp

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [19]:
# Try 2 that works
from transformers import pipeline
translator = {}
translator['fr_en'] = pipeline("translation", model='Helsinki-NLP/opus-mt-fr-en')
translator['fr_en']("Ce cours est produit par Hugging Face.")

# translator['en_fr'] = pipeline("translation", model=model_translation['en_fr'])
# translator['en_fr']("My name is Mathieu and I like computers.")


loading configuration file config.json from cache at /home/desponds/.cache/huggingface/hub/models--Helsinki-NLP--opus-mt-fr-en/snapshots/49463f1706007cb314a942296b77a6483e6f6953/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-fr-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "decoder_vocab_size": 59514,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkp

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [ ]:
def translate(example, translation):
    example["traduction"] = translator[translation](example['text'])
    return example

translated_dataset = dataset_fr.map(translate, 'fr_en')